# VacationPy
----

#### My Observations
* ....

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
data = pd.read_csv('../OutputData/cities.csv')
data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,49.84,78,30,7.07,NZ,1594435433
1,Saldanha,-33.01,17.94,51.80,81,75,12.75,ZA,1594435451
2,Arraial do Cabo,-22.97,-42.03,69.55,76,32,16.62,BR,1594435425
3,Bredasdorp,-34.53,20.04,50.00,81,70,16.11,ZA,1594435434
4,Tuktoyaktuk,69.45,-133.04,42.80,87,75,12.75,CA,1594435452
...,...,...,...,...,...,...,...,...,...
537,Ocós,14.51,-92.19,75.20,100,40,2.24,GT,1594435613
538,Barentu,15.11,37.59,73.67,74,3,5.82,ER,1594435614
539,Owando,-0.48,15.90,71.55,95,100,2.66,CG,1594435614
540,Chara,56.91,118.26,85.55,29,1,17.49,RU,1594435614


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key = g_key)

In [10]:
lat_long = data[['Latitude', 'Longitude']]
lat_long

,Latitude,Longitude
0,-46.19,168.86
1,-33.01,17.94
2,-22.97,-42.03
3,-34.53,20.04
4,69.45,-133.04
...,...,...
537,14.51,-92.19
538,15.11,37.59
539,-0.48,15.90
540,56.91,118.26


In [11]:
humidity = data['Humidity'].astype(int)
humidity

0       78
1       81
2       76
3       81
4       87
      ... 
537    100
538     74
539     95
540     29
541     72
Name: Humidity, Length: 542, dtype: int32

In [12]:
type(humidity[0])

numpy.int32

In [13]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=10, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
weather_crit = data[['City', 'Country', 'Latitude', 'Longitude', 'Max Temperature', 'Cloudiness', 'Wind Speed']]
weather_crit

,City,Country,Latitude,Longitude,Max Temperature,Cloudiness,Wind Speed
0,Mataura,NZ,-46.19,168.86,49.84,30,7.07
1,Saldanha,ZA,-33.01,17.94,51.80,75,12.75
2,Arraial do Cabo,BR,-22.97,-42.03,69.55,32,16.62
3,Bredasdorp,ZA,-34.53,20.04,50.00,70,16.11
4,Tuktoyaktuk,CA,69.45,-133.04,42.80,75,12.75
...,...,...,...,...,...,...,...
537,Ocós,GT,14.51,-92.19,75.20,40,2.24
538,Barentu,ER,15.11,37.59,73.67,3,5.82
539,Owando,CG,-0.48,15.90,71.55,100,2.66
540,Chara,RU,56.91,118.26,85.55,1,17.49


In [24]:
weather_crit['Hotel Name']=""
weather_crit

C:\Users\mgban\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Country,Latitude,Longitude,Max Temperature,Cloudiness,Wind Speed,Hotel Name
0,Mataura,NZ,-46.19,168.86,49.84,30,7.07,
1,Saldanha,ZA,-33.01,17.94,51.80,75,12.75,
2,Arraial do Cabo,BR,-22.97,-42.03,69.55,32,16.62,
3,Bredasdorp,ZA,-34.53,20.04,50.00,70,16.11,
4,Tuktoyaktuk,CA,69.45,-133.04,42.80,75,12.75,
...,...,...,...,...,...,...,...,...
537,Ocós,GT,14.51,-92.19,75.20,40,2.24,
538,Barentu,ER,15.11,37.59,73.67,3,5.82,
539,Owando,CG,-0.48,15.90,71.55,100,2.66,
540,Chara,RU,56.91,118.26,85.55,1,17.49,


In [25]:
ideal_weather = weather_crit.loc[(weather_crit['Max Temperature'] > 70) & 
                                 (weather_crit['Max Temperature'] < 80) & 
                                 (weather_crit['Wind Speed'] < 10) & 
                                 (weather_crit['Cloudiness'] == 0)]
ideal_weather

,City,Country,Latitude,Longitude,Max Temperature,Cloudiness,Wind Speed,Hotel Name
106,Awjilah,LY,29.11,21.29,72.18,0,8.61,
113,Mednogorsk,RU,51.41,57.59,79.95,0,1.97,
141,Marsá Maţrūḩ,EG,31.35,27.25,71.60,0,5.82,
214,Jalu,LY,29.03,21.55,72.28,0,8.10,
225,Komsomolets,KZ,53.75,62.07,79.97,0,4.27,
302,Corinth,GR,37.94,22.96,74.52,0,1.01,
360,Al Bardīyah,LY,31.76,25.09,71.01,0,8.59,
371,Lazurne,UA,46.08,32.53,72.57,0,7.45,
378,Derzhavīnsk,KZ,51.10,66.32,76.37,0,7.54,
444,Ben Gardane,TN,33.14,11.22,72.41,0,5.01,


In [26]:
ideal_weather.count()

City               10
Country            10
Latitude           10
Longitude          10
Max Temperature    10
Cloudiness         10
Wind Speed         10
Hotel Name         10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
